In [1]:
!pip3 install fastFM

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastFM: filename=fastFM-0.2.10-cp310-cp310-linux_x86_64.whl size=591784 sha256=6fcb1d7e7be1f71dcd0e334248b768c846881bfd9a96609426e26b41d064b35a
  Stored in directory: /root/.cache/pip/wheels/93/92/52/2da7997fcb7a7ce9042ff3b33836ef0c2fd47aa95382d7a113
Successfully built fastFM


In [3]:
import os
import numpy as np
import pandas as pd
import fastFM
from fastFM.datasets import make_user_item_regression
from sklearn.model_selection import train_test_split
from fastFM import sgd
from fastFM import als
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix
from fastFM import mcmc
import functools as fct
import itertools as itools
import random, scipy

In [4]:
train_df = pd.read_csv("training_set.csv")
train_df.head()

,userID,itemID,styleID,rating
0,401,32780,12224,3.0
1,7548,21688,9020,3.5
2,7663,1521,568,4.0
3,1357,3824,1417,3.5
4,1361,727,263,4.5


## Convertir a formato fastFM

In [9]:
def get_single_entries_in_fm_input_format(data, itemlist):

    '''Cree el formato de entrada necesario (datos, (fila, columna)) para la matriz csc para
    las entradas individuales en los datos. Cada entrada ocuparía una fila. Esto significa que
    daría como resultado una matriz csc con dimensión (| datos | x | lista de elementos |).
    '''

    column = len(itemlist)
    row = len(data)
    shape = (row, column)

    row_inds = np.zeros(len(data), dtype=np.int32)
    col_inds = np.zeros(len(data), dtype=np.int32)
    datalist = np.zeros(len(data), dtype=np.float32)

    for i in range(len(data)):
        item = data[i]
        val = 1
        datalist[i] = val

        # ubica su posición en la lista de elementos, arroja un error si el elemento no es un
        # artículo posible
        col_ind = np.where(itemlist==item)[0]

        # no deben ser elementos duplicados en la lista de elementos
        assert len(col_ind) == 1
        col_ind = col_ind[0]
        row_ind = i

        col_inds[i] = col_ind
        row_inds[i] = row_ind

    return datalist, row_inds, col_inds, shape


def get_multi_entries_in_fm_input_format(data, itemlist, norm_func=None):

    '''Cree el formato de entrada necesario (datos, (fila, columna)) para la matriz csc para
    las entradas múltiples en los datos. Cada conjunto de entradas múltiples ocuparía una fila.
    Esto significa que daría como resultado una matriz csc con dimensión
    (| conjuntos de entradas en datos | x | lista de elementos |).
    '''

    column = len(itemlist)

    # número de conjuntos de entradas en los datos
    row = len(data)
    shape = (row, column)

    # numero de datos
    num_of_data = fct.reduce(lambda x, y: x + len(y), data, 0)
    row_inds = np.zeros(num_of_data, dtype=np.int32)
    col_inds = np.zeros(num_of_data, dtype=np.int32)
    datalist = np.zeros(num_of_data, dtype=np.float32)
    cnt = 0
    for i in range(len(data)):
        multi_entry = data[i]

        if norm_func != None:
            # función que recibe el tamaño del multi_entry para decidir cómo normalizarlo
            val = norm_func(len(multi_entry))
        else:
            # asignación de valor binario por defecto
            val = 1 if len(multi_entry) > 0 else 0

        # para cada entrada en multi_entry, ubique su posición en la lista de elementos,
        # arroja error si el elemento no es un elemento posible
        # todas las entradas permanecen en la misma fila
        row_ind = i
        for item in multi_entry:
            col_ind = np.where(itemlist==item)[0]
            assert len(col_ind) == 1
            col_ind = col_ind[0]

            datalist[cnt] = val
            col_inds[cnt] = col_ind
            row_inds[cnt] = row_ind

            # actualiza contador
            cnt += 1

    return datalist, row_inds, col_inds, shape

### Conversión de los datos

In [10]:
beerlist = train_df.sort_values('itemID')['itemID'].unique()
userlist = train_df.sort_values('userID')['userID'].unique()
stylelist = train_df.sort_values('styleID')['styleID'].unique()

# usuarios que dieron ratings
user_data = train_df['userID'].values

# items que recibieron ratings
beer_data = train_df['itemID'].values

# data de estilo de cerveza
styles_data = train_df['styleID'].values

# target vector: ratings
rating_data = train_df['rating'].values


# convertir a formato fastFM utilizando funciones de arriba
user_datalist, user_row_inds, user_col_inds, user_shape = get_single_entries_in_fm_input_format(data=user_data,
                                                                                                itemlist=userlist)

beer_datalist, beer_row_inds, beer_col_inds, beer_shape = get_single_entries_in_fm_input_format(data=beer_data,
                                                                                                   itemlist=beerlist)

style_datalist, style_row_inds, style_col_inds, style_shape = get_single_entries_in_fm_input_format(data=styles_data,
                                                                                                   itemlist=stylelist)

# Concatena las dos columnas cambiando los índices de las columnas relacionadas con beer.
# cambiar por el número de columnas en las columnas de usuario
shift_by = len(userlist)
beer_col_inds += shift_by
beer_col_inds += shift_by

# concatena los datos (agregamos item_styles)
datalist = np.append(user_datalist, [beer_datalist, style_datalist])
row_inds = np.append(user_row_inds, [beer_row_inds, style_row_inds])
col_inds = np.append(user_col_inds, [beer_col_inds,style_col_inds])

# asegúrese de que ambos conjuntos de características tengan el mismo número de filas
print('User feature set shape: {}\nItem feature set shape: {}\nStyle feature set shape: {}'.format(user_shape, beer_shape, style_shape))

assert user_shape[0] == beer_shape[0]
shape = (user_shape[0], user_shape[0] + beer_shape[0] + style_shape[0])
print('Dimension of FM input: {}'.format(shape))

X = csc_matrix((datalist, (row_inds, col_inds)), shape=shape)
y = rating_data

User feature set shape: (35534, 8320)
Item feature set shape: (35534, 1692)
Style feature set shape: (35534, 202)
Dimension of FM input: (35534, 106602)
